# %% [markdown]
# ## Chap 2: PyTorch & KORNIA 入門
# このノートでは、PyTorch と Kornia を使って画像を読み込み、前処理し、特徴点を検出する基本を学びます。
# 
# サンプル画像パス:
# `C:\Users\337587\PlayGround\01_Personal\Kaggle\image-matching-challenge-2025\data\train\imc2024_lizard_pond\lizard_00003.png`

# %% [markdown]
# ### 1. ライブラリのインポートとデバイス設定


In [2]:

# %%
import os
import numpy as np
import torch
import kornia as K
import matplotlib.pyplot as plt
from PIL import Image

# PyTorch: GPU が使えるか確認し、device を設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# %% [markdown]


Using device: cuda


In [ ]:
# ### 2. 画像の読み込み・表示
# - PIL で開き、NumPy 配列に変換
# - Kornia のユーティリティで Tensor に変換／表示

# %%
# パスの設定
img_path = r"C:\Users\337587\PlayGround\01_Personal\Kaggle\image-matching-challenge-2025\data\train\imc2024_lizard_pond\lizard_00003.png"
# PIL で読み込み
img_pil = Image.open(img_path).convert('RGB')
# NumPy 配列 (H, W, C)
img_np = np.array(img_pil)

# NumPy を Tensor (C, H, W) に変換し、0-1 正規化
from kornia.utils import image_to_tensor, tensor_to_image
img_tensor = image_to_tensor(img_np, keepdim=False).float() / 255.0  # shape [3, H, W]
img_tensor = img_tensor.to(device)

# Matplotlib で表示 (Tensor -> NumPy)
plt.figure(figsize=(6,6))
plt.imshow(tensor_to_image(img_tensor))
plt.axis('off')
plt.title('Original Image')
plt.show()


In [ ]:

# %% [markdown]
# ### 3. 基本的な前処理: グレースケール化 & リサイズ

# %%
# グレースケール化 (shape [1, H, W])
gray_tensor = K.color.rgb_to_grayscale(img_tensor)
# リサイズ (PyTorch Tensor のみ)
resized_tensor = K.geometry.transform.resize(img_tensor.unsqueeze(0), (256, 256)).squeeze(0)

# 表示: グレースケール
plt.figure(figsize=(5,5))
plt.imshow(tensor_to_image(gray_tensor), cmap='gray')
plt.axis('off')
plt.title('Grayscale Image')
plt.show()

# 表示: リサイズ
plt.figure(figsize=(5,5))
plt.imshow(tensor_to_image(resized_tensor))
plt.axis('off')
plt.title('Resized to 256×256')
plt.show()


In [ ]:

# %% [markdown]
# ### 4. エッジ検出 (Canny フィルタ)

# %%
# Kornia の Canny
canny = K.filters.Canny(low_threshold=0.1, high_threshold=0.2)
# Canny に入れるには [B, 1, H, W]
edges = canny(gray_tensor.unsqueeze(0))[0][0]

plt.figure(figsize=(5,5))
plt.imshow(edges.cpu().numpy(), cmap='gray')
plt.axis('off')
plt.title('Canny Edges')
plt.show()


In [ ]:

# %% [markdown]
# ### 5. 特徴点検出 (Harris 角検出)
# - Harris コーナー反応関数を計算し、ピークを抽出

# %%
# Harris 反応マップ [B, 1, H, W]
harris_response = K.feature.harris_response(gray_tensor.unsqueeze(0), k=0.04)
# 上位 50 点のピーク位置を取得
coords = K.feature.corner_peaks(harris_response, num_peaks=50, threshold=0.01)
# coords は [B, N, 2] (y, x)
points = coords[0].cpu().numpy()

# キーポイントを描画
from kornia.utils import draw_keypoints
# draw_keypoints: 入力は [B, C, H, W]、coords 正規化不要
vis = draw_keypoints(img_tensor.unsqueeze(0), coords, r=3)

plt.figure(figsize=(6,6))
plt.imshow(tensor_to_image(vis.squeeze(0)))
plt.axis('off')
plt.title('Harris Keypoints')
plt.show()



# %% [markdown]
# ## 次のステップ
# - 特徴記述子 (SIFT/ORB) を使った記述子抽出
# - コサイン距離／kNN によるマッチング
# - 小規模なペアで検出→マッチング→可視化を試してみましょう。
